<a href="https://colab.research.google.com/github/Giandonn/Machine-Learning/blob/main/TrabalhoRedesNeuraisGraficos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Carrega o dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalização
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encoding dos rótulos
y_train, y_test = to_categorical(y_train), to_categorical(y_test)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
param_space = {
    'num_filters': [32, 64, 128],
    'kernel_size': [3, 5],
    'dropout_rate': [0.25, 0.5],
    'dense_units': [64, 128],
    'optimizer': ['adam', 'sgd']
}

def create_individual():
    return {
        'num_filters': random.choice(param_space['num_filters']),
        'kernel_size': random.choice(param_space['kernel_size']),
        'dropout_rate': random.choice(param_space['dropout_rate']),
        'dense_units': random.choice(param_space['dense_units']),
        'optimizer': random.choice(param_space['optimizer'])
    }


In [4]:
def build_model(ind):
    model = models.Sequential([
        layers.Conv2D(ind['num_filters'], (ind['kernel_size'], ind['kernel_size']), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(ind['dropout_rate']),
        layers.Dense(ind['dense_units'], activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    opt = optimizers.Adam() if ind['optimizer'] == 'adam' else optimizers.SGD()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [5]:
def fitness(ind):
    model = build_model(ind)
    history = model.fit(
        x_train[:10000], y_train[:10000],
        epochs=3, batch_size=64,
        verbose=0, validation_split=0.2
    )
    return history.history['val_accuracy'][-1]


In [6]:
def crossover(p1, p2):
    return {k: random.choice([p1[k], p2[k]]) for k in p1}

def mutate(ind):
    key = random.choice(list(param_space.keys()))
    ind[key] = random.choice(param_space[key])
    return ind


In [ ]:
pop_size = 4
num_generations = 3
mutation_rate = 0.3

population = [create_individual() for _ in range(pop_size)]
history_geracoes = []

for generation in range(num_generations):
    print(f"\nGeração {generation+1}")
    scores = [fitness(ind) for ind in population]

    history_geracoes.append({
        'scores': scores.copy(),
        'best_accuracy': max(scores),
        'avg_accuracy': np.mean(scores),
        'worst_accuracy': min(scores)
    })

    for i, (ind, score) in enumerate(zip(population, scores)):
        print(f"Indivíduo {i+1}: {ind}, Acurácia: {score:.4f}")

    sorted_population = [x for _, x in sorted(zip(scores, population), key=lambda pair: pair[0], reverse=True)]
    population = sorted_population[:2]

    while len(population) < pop_size:
        p1, p2 = random.sample(population[:2], 2)
        child = crossover(p1, p2)
        if random.random() < mutation_rate:
            child = mutate(child)
        population.append(child)

print("\nMelhor modelo final:")
best_model = population[0]
print(best_model)



Geração 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Indivíduo 1: {'num_filters': 128, 'kernel_size': 3, 'dropout_rate': 0.5, 'dense_units': 128, 'optimizer': 'sgd'}, Acurácia: 0.2850
Indivíduo 2: {'num_filters': 128, 'kernel_size': 3, 'dropout_rate': 0.5, 'dense_units': 64, 'optimizer': 'sgd'}, Acurácia: 0.3135
Indivíduo 3: {'num_filters': 64, 'kernel_size': 3, 'dropout_rate': 0.25, 'dense_units': 128, 'optimizer': 'adam'}, Acurácia: 0.4960
Indivíduo 4: {'num_filters': 128, 'kernel_size': 3, 'dropout_rate': 0.5, 'dense_units': 128, 'optimizer': 'adam'}, Acurácia: 0.5120

Geração 2


In [ ]:
sns.set(style="whitegrid")

geracoes = list(range(1, num_generations + 1))
melhores = [h['best_accuracy'] for h in history_geracoes]
medias = [h['avg_accuracy'] for h in history_geracoes]
piores = [h['worst_accuracy'] for h in history_geracoes]

plt.figure(figsize=(12, 6))
plt.plot(geracoes, melhores, label='Melhor da Geração', marker='o', linewidth=2, color='green')
plt.plot(geracoes, medias, label='Média da Geração', marker='x', linewidth=2, color='blue')
plt.plot(geracoes, piores, label='Pior da Geração', marker='s', linewidth=2, color='red')
plt.xlabel("Geração")
plt.ylabel("Acurácia de Validação")
plt.title("Evolução da Acurácia por Geração (CIFAR-10 com Algoritmo Genético)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
data_box = [h['scores'] for h in history_geracoes]
sns.boxplot(data=data_box)
plt.xticks(ticks=np.arange(num_generations), labels=[f"Geração {i+1}" for i in range(num_generations)])
plt.ylabel("Acurácia")
plt.title("Distribuição das Acurácias por Geração")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()